# Load data from one file

In [84]:
# pip install fsspec / conda install -c conda-forge fsspec
# pip install s3fs
import pandas as pd
import numpy as np
import boto3
import os
from sagemaker import get_execution_role

role = get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [88]:
bucket_name = 'eliezerraj-908671954593-dataset'
prefix_name = 'payment'
file_name = 'payment.csv'

data_location = 's3://{}/{}/{}'.format(bucket_name, prefix_name, file_name)

df = pd.read_csv(data_location)

print("---------------------------------")
print(f"Shape of dataframe '{df.shape}':")

---------------------------------
Shape of dataframe '(10130, 25)':


In [90]:
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
df.head()

,id,fk_card_id,card_number,terminal_name,coord_x,coord_y,card_type,card_model,payment_at,mcc,amount,night_day,ic_night_day,wkend_wkday,ic_wkend_wkday,day_of_year,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_tx,time_btw_cc_tx,fraud
0,1,34,111.111.000.034,TERM-186,151.0,115.0,CREDIT,VIRTUAL,2024-04-01 09:06:29.000 -0300,COMPUTER,2088.0,day,0,wkday,0,92,1,2088.0,1,2088.0,1,2088.0,565,0,1
1,2,391,111.111.000.391,TERM-17,34.0,2.0,CREDIT,VIRTUAL,2024-04-01 09:15:54.000 -0300,BOOK,109.0,day,0,wkday,0,92,1,109.0,1,109.0,1,109.0,102,0,0
2,3,865,111.111.000.865,TERM-92,69.0,36.0,CREDIT,VIRTUAL,2024-04-01 09:17:36.000 -0300,STORE,394.0,day,0,wkday,0,92,1,394.0,1,394.0,1,394.0,194,0,0
3,4,233,111.111.000.233,TERM-47,3.0,65.0,CREDIT,CHIP,2024-04-01 09:20:50.000 -0300,COMPUTER,664.0,day,0,wkday,0,92,2,395.5,2,395.5,2,395.5,466,0,0
4,5,75,111.111.000.075,TERM-90,97.0,77.0,CREDIT,VIRTUAL,2024-04-01 09:28:36.000 -0300,PARKING,47.0,day,0,wkday,0,92,2,145.5,2,145.5,2,145.5,505,0,0


# Load data from a chunk of files in a folder

In [80]:
s3 = boto3.client('s3')

bucket_name = 'eliezerraj-908671954593-dataset'
prefix_name = 'payment/glue-job/'

# List objects in the specified bucket and prefix
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix_name)

# Extract the list of file keys from the response
file_keys = [obj['Key'] for obj in response.get('Contents', [])]

In [81]:
# Read each file
df = pd.DataFrame()

for obj_key in file_keys[1:]:
    # Read the object content
    obj_content = s3.get_object(Bucket=bucket_name, Key=obj_key)['Body'].read()
    print(f"Content of file '{obj_key}':")

    data_location = 's3://{}/{}'.format(bucket_name, obj_key)

    df_temp = pd.read_csv(data_location)
    df = pd.concat([df, df_temp], ignore_index=True)
    print(f"Shape of chunk '{df.shape}':")
    print("---------------------------------")

print("---------------------------------")
print(f"Shape of dataframe '{df.shape}':")

Content of file 'payment/glue-job/run-1713493427814-part-r-00000':
Shape of chunk '(2589, 25)':
---------------------------------
Content of file 'payment/glue-job/run-1713493427814-part-r-00001':
Shape of chunk '(5179, 25)':
---------------------------------
Content of file 'payment/glue-job/run-1713493427814-part-r-00002':
Shape of chunk '(7769, 25)':
---------------------------------
Content of file 'payment/glue-job/run-1713493427814-part-r-00003':
Shape of chunk '(10359, 25)':
---------------------------------
Content of file 'payment/glue-job/run-1713493427814-part-r-00004':
Shape of chunk '(12949, 25)':
---------------------------------
Content of file 'payment/glue-job/run-1713493427814-part-r-00005':
Shape of chunk '(15539, 25)':
---------------------------------
Content of file 'payment/glue-job/run-1713493427814-part-r-00006':
Shape of chunk '(18129, 25)':
---------------------------------
Content of file 'payment/glue-job/run-1713493427814-part-r-00007':
Shape of chunk '(20

In [82]:
df.head(3)

,id,fk_card_id,card_number,terminal_name,coord_x,coord_y,card_type,card_model,payment_at,mcc,amount,night_day,ic_night_day,wkend_wkday,ic_wkend_wkday,day_of_year,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_tx,time_btw_cc_tx,fraud
0,26965,843,111.111.000.843,TERM-221,222.0,261.0,CREDIT,VIRTUAL,2024-03-19 21:03:38.0,GAS,840.0,night,1,wkday,0,79.0,4,857.25,6,659.17,14,408.07,63,26,1
1,37036,636,111.111.000.636,TERM-59,38.0,34.0,CREDIT,VIRTUAL,2024-04-18 18:57:19.0,LAUNDRY,142.0,day,0,wkday,0,109.0,1,142.00,3,274.67,10,254.30,149,0,0
2,28164,538,111.111.000.538,TERM-24,77.0,45.0,CREDIT,CHIP,2024-03-23 15:47:16.0,SPORTING,408.0,day,0,wkend,1,83.0,3,209.33,3,209.33,6,207.17,8,1802,0


# featuring enginering

In [69]:
df = df.filter(['fraud',
                'payment_at',
                'terminal_name',
                'coord_x',
                'coord_y',
                'card_type',
                'card_model',
                'mcc',
                'amount',
                'tx_1d',
                'avg_1d',
                'tx_7d',
                'avg_7d',
                'tx_30d',
                'avg_30d',
                'time_btw_cc_tx'], axis=1)

In [70]:
df.head(3)

,fraud,payment_at,terminal_name,coord_x,coord_y,card_type,card_model,mcc,amount,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_cc_tx
0,1,2024-03-19 21:03:38.0,TERM-221,222.0,261.0,CREDIT,VIRTUAL,GAS,840.0,4,857.25,6,659.17,14,408.07,26
1,0,2024-04-18 18:57:19.0,TERM-59,38.0,34.0,CREDIT,VIRTUAL,LAUNDRY,142.0,1,142.00,3,274.67,10,254.30,0
2,0,2024-03-23 15:47:16.0,TERM-24,77.0,45.0,CREDIT,CHIP,SPORTING,408.0,3,209.33,3,209.33,6,207.17,1802


In [71]:
# Create a new column distance
import math

df.insert(5, "distance", 0)
for ind in df.index:
    p1 = [0, 0]
    p2 = [df['coord_x'][ind], df['coord_y'][ind]]
    distance = math.sqrt(((p1[0] - p2[0]) ** 2) + ((p1[1] - p2[1]) ** 2))
    df["distance"][ind] = distance

/tmp/ipykernel_216/2901702902.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["distance"][ind] = distance
/tmp/ipykernel_216/2901702902.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '342.64413025761877' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df["distance"][ind] = distance


In [72]:
# extract hour
df["ts_payment_at"] = pd.to_datetime(df['payment_at'])
print(df.ts_payment_at.dtype)
df["hour"] = df.ts_payment_at.dt.hour

datetime64[ns]


In [73]:
df = df.filter(['fraud',
                'terminal_name',
                'hour',
                'distance',
                'card_type',
                'card_model',
                'mcc',
                'amount',
                'tx_1d',
                'avg_1d',
                'tx_7d',
                'avg_7d',
                'tx_30d',
                'avg_30d',
                'time_btw_cc_tx'], axis=1)

In [74]:
df.head(3)

,fraud,terminal_name,hour,distance,card_type,card_model,mcc,amount,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_cc_tx
0,1,TERM-221,21,342.644130,CREDIT,VIRTUAL,GAS,840.0,4,857.25,6,659.17,14,408.07,26
1,0,TERM-59,18,50.990195,CREDIT,VIRTUAL,LAUNDRY,142.0,1,142.00,3,274.67,10,254.30,0
2,0,TERM-24,15,89.185201,CREDIT,CHIP,SPORTING,408.0,3,209.33,3,209.33,6,207.17,1802


In [75]:
df = pd.get_dummies(df, columns=['card_model', ], prefix='card_model', dtype=int, drop_first=False)
df = pd.get_dummies(df, columns=['card_type', ], prefix='card_type', dtype=int, drop_first=False)
df.head(3)

,fraud,terminal_name,hour,distance,mcc,amount,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_cc_tx,card_model_CHIP,card_model_VIRTUAL,card_type_CREDIT
0,1,TERM-221,21,342.644130,GAS,840.0,4,857.25,6,659.17,14,408.07,26,0,1,1
1,0,TERM-59,18,50.990195,LAUNDRY,142.0,1,142.00,3,274.67,10,254.30,0,0,1,1
2,0,TERM-24,15,89.185201,SPORTING,408.0,3,209.33,3,209.33,6,207.17,1802,1,0,1


In [76]:
df = df.filter(['fraud',
                'distance',
                'card_model_CHIP',
                'card_model_VIRTUAL',
                'card_type_CREDIT',
                'amount',
                'tx_1d',
                'avg_1d',
                'tx_7d',
                'avg_7d',
                'tx_30d',
                'avg_30d',
                'time_btw_cc_tx'], axis=1)

In [77]:
df.head(3)

,fraud,distance,card_model_CHIP,card_model_VIRTUAL,card_type_CREDIT,amount,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_cc_tx
0,1,342.644130,0,1,1,840.0,4,857.25,6,659.17,14,408.07,26
1,0,50.990195,0,1,1,142.0,1,142.00,3,274.67,10,254.30,0
2,0,89.185201,1,0,1,408.0,3,209.33,3,209.33,6,207.17,1802
